In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

data = pd.read_csv("/content/drive/My Drive/Tugas Akhir/dataset_with_spelling_check.csv")

In [ ]:
#Melakukan tokenize berdasarkan white space
from nltk.tokenize import RegexpTokenizer

data['komentar'] = data['komentar'].apply(str)

tokenizer = RegexpTokenizer('\s+', gaps=True)
comments = []
for comment in data['komentar']:
    comments.append(tokenizer.tokenize(comment))

NameError: ignored

In [ ]:
#Melakukan tokenize berdasarkan white space
from nltk.tokenize import RegexpTokenizer

text = "makanya rezim jokowi minta tunda pemilu biar bisa utang lagi gampang koq"
tokenizer = RegexpTokenizer('\s+', gaps=True)
token= tokenizer.tokenize(text)

In [ ]:
token

['makanya',
 'rezim',
 'jokowi',
 'minta',
 'tunda',
 'pemilu',
 'biar',
 'bisa',
 'utang',
 'lagi',
 'gampang',
 'koq']

In [ ]:
comments[:10]

NameError: ignored

In [ ]:
#Filter dari invalid caracter
invalid = '�'
for i in range(0, len(comments)):
    comment = comments[i]
    for j in range(0, len(comment)):
        sub_comment = comments[i][j]
        for k in sub_comment:
            if k in invalid:
                comments[i][j] = comments[i][j].replace(k, "")

NameError: ignored

In [ ]:
#Menghilangkan token yang kosong
for comment in comments:
    for sub_comment in comment:
        if sub_comment == '':
            comment.remove(sub_comment)

In [ ]:
comments[:5]

In [ ]:
pip install --upgrade gensim

In [ ]:
pip install --upgrade numpy

In [ ]:
from gensim.models import Word2Vec
path = F"/content/drive/My Drive/Tugas Akhir/idwiki_word2vec_200_new_lower.model"
model = Word2Vec.load(path)

In [ ]:
#Memproses data slang
slang_corpus = pd.read_csv("/content/drive/My Drive/Tugas Akhir/slang.csv")
slang_corpus

In [ ]:
#Filter data slang
slang_corpus = slang_corpus[slang_corpus.columns[0:2]]
slang_corpus

In [ ]:
import gensim
print(gensim.__version__)

In [ ]:
#Menggunakan perbandingan dengan word2vec yang ada dan edit distance untuk membuat kata menjadi kata formal
import time
from datetime import timedelta
print("Processing data...")
start_time = time.time()
import nltk
corr_comments = []
slang_words = []
reco_words = []
corr_word = 0
tot_word = 0
for comment in comments:
    corr_comment = []
    slang_word = []
    for sub_comment in comment:
        if sub_comment in model.wv.index_to_key:
            sim_slang = slang_corpus[slang_corpus['slang'] == sub_comment]
            if len(sim_slang) != 0:
                mode_word = sim_slang.formal.mode()[0]
                corr_word = corr_word + 1
                corr_comment.append(mode_word)
            else:
                corr_word = corr_word + 1
                corr_comment.append(sub_comment)
        else:
            comment_found = False
            sim_slang = slang_corpus[slang_corpus['slang'] == sub_comment]
            if len(sim_slang) != 0:
                mode_word = sim_slang.formal.mode()[0]
                corr_word = corr_word + 1
                corr_comment.append(mode_word)
                comment_found = True
            else:
                rat_words = []
                for slang in slang_corpus['slang']:
                    rat_word = nltk.edit_distance(sub_comment, slang)
                    rat_words.append(rat_word)
                min_rat = rat_words.index(min(rat_words))
                value = str(slang_corpus.formal.loc[min_rat])
                corr_words = corr_word + 1
                corr_comment.append(value)
                slang_words.append(sub_comment + " = " + value)
                comment_found = True
            if comment_found ==  False:
                corr_comment.append(sub_comment)
                slang_words.append(sub_comment + " = " + "Not Found")
        tot_word = tot_word + 1
    corr_comments.append(corr_comment)
    print("Total kata yang sudah diproses : " + str(tot_word))
print("Total kata yang diperbaiki : " + str(corr_word) + " dari total semua kata : " + str(tot_word))
finish_time = time.time()
print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

In [ ]:
data['komentar'] = corr_comments

In [ ]:
def join(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text

In [ ]:
data['komentar'] = data['komentar'].apply(lambda x: join(x))
data

In [ ]:
data['komentar''] = data['komentar''].str.lower()

In [ ]:
data.to_csv("dataset_sc.csv")  